Задание 1: Анализ финансовых транзакций и клиентов.

Описание:

В тестовом задании вам предоставляются два набора данных:
Файл transactions_data.xlsx (данные о финансовых транзакциях):
•	transaction_id – уникальный идентификатор транзакции
•	client_id – идентификатор клиента, совершившего транзакцию
•	transaction_date – дата проведения транзакции
•	service – тип оказанной финансовой услуги
•	amount – сумма транзакции (выручка)
•	payment_method – способ оплаты (банковский перевод, кредитная карта и т. д.)
•	city – город, в котором была совершена транзакция
•	consultant – имя финансового консультанта, оформившего транзакцию

Файл clients_data.json (данные о клиентах):
•	client_id – уникальный идентификатор клиента
•	age – возраст клиента
•	gender – пол клиента
•	net_worth – чистая стоимость активов клиента

Данные могут содержать пропуски, ошибки (например, некорректные даты), а также аномальные значения (отрицательные суммы, несуществующие услуги и т. д.), поэтому перед анализом необходимо их очистить и нормализовать.

Задачи:
1.	Очистка и подготовка данных. Проверить данные на пропущенные и аномальные значения (например, отрицательные суммы транзакций). Удалить или исправить некорректные данные. Привести формат дат к единому стандарту.

In [74]:
import pandas as pd

In [75]:
#Чтение файлов:
df1T = pd.read_excel('Источники/transactions_data.xlsx')
df2C = pd.read_json('Источники/clients_data.json')  

In [76]:
#Проверки на дубли:
df1T.duplicated().sum()
df2C.duplicated().sum()

np.int64(0)

In [89]:
#Проверка на пустые значения:
df1T.isna().sum() #Проверка на Пустые значения в Транзакциях. 

#UPD: Значения по 0 должно быть, после очистки. UPD: Есть данные где остальные присутствуют, но нет дат. 

transaction_id        0
client_id             0
transaction_date    805
service               0
amount                0
payment_method        0
city                  0
consultant            0
dtype: int64

In [78]:
#Проверка на пустые значения:
df2C.isna().sum() #Проверка на Пустые значения в Клиентах.

id              1
age           983
gender       3319
net_worth     475
dtype: int64

In [79]:
#Очистка данных на пустые значения файла Транзакции:
df1T = df1T.dropna() 

#Провоерка выше.

In [80]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

Пробуем сначала преобразовать даты, есть в строках со значением "INVALID_DATE". 
Возможно действие не эффективно и не целесообразное, но мы проверим. 

In [90]:
#Преобразование дат к одному в формату с целью проверки изменения значения:
df1T['transaction_date'] = pd.to_datetime(df1T['transaction_date'])
#Приведение всех дат к одному формату с точностью до секунд:
df1T['transaction_date'] = df1T['transaction_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

#*Чтобы сработало нужно ниже выполнить сначала "#Замена INVALID_DATE на Nan".

Нам это дало ошибку, заменим на пустые значения, проверим, может есть данные где нет дат,
но остальные данные целостные. 

Unknown datetime string format, unable to parse: INVALID_DATE, at position 15

In [82]:
#Замена INVALID_DATE на Nan

df1T['transaction_date'] = pd.to_datetime(df1T['transaction_date'], errors='coerce')

In [84]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

Есть файлы где нет даты, но остальные в норме. Продолжим удаление, но по условиям. 

In [85]:
#Удаление строк с "Неизвестной услугой":
df1T = df1T[df1T['service'] != 'Неизвестная услуга']

In [91]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

In [97]:
#Продолжаем очистку, теперь зайдем со стороны аномальных значений:
#Использую "Межквартильный размах" (IQR). 

# Рассчитываем первый и третий квартели, чтобы посчитать IQR.
Q1 = df1T['amount'].quantile(0.25)
Q3 = df1T['amount'].quantile(0.75)

#Расчитываем IQR:
IQR = Q3 - Q1

#Устанавливаем границы 
lower_bound = Q1 - 1.5 * IQR #Нижняя.
upper_bound = Q3 + 1.5 * IQR #Верхняя.

#Вывод границ: 
print(lower_bound)
print(upper_bound)

-58592.28318671013
149490.98131813295


In [102]:
#Вывод аномальных значений из файла:
anomal = df1T[(df1T['amount'] < lower_bound) | (df1T['amount'] > upper_bound)]
print(anomal[['service','amount']])

                              service         amount
65                Управление активами  165531.626439
191   Инвестиционное консультирование  176346.818037
251   Инвестиционное консультирование  158548.065675
272               Управление активами  163445.690198
347           Финансовое планирование  194496.625320
...                               ...            ...
9500        Структурирование капитала  158543.668647
9631        Структурирование капитала  168401.012948
9643           Налоговое планирование  160444.593254
9671  Инвестиционное консультирование  161210.845033
9743              Управление активами  154649.101918

[151 rows x 2 columns]


In [104]:
#Удаление аномальных значений: 
df1T = df1T[(df1T['amount'] >= lower_bound) & (df1T['amount'] <= upper_bound)] 

In [106]:
#Сохранение данных:
df1T.to_excel('Источники/transactions_data.xlsx', index=False)

In [108]:
#Проверка: 
#Вывод аномальных значений из файла:
anomal = df1T[(df1T['amount'] < lower_bound) | (df1T['amount'] > upper_bound)]

print(anomal)

Empty DataFrame
Columns: [transaction_id, client_id, transaction_date, service, amount, payment_method, city, consultant]
Index: []
